In [ ]:
#การนำเข้าข้อมูล โดย import และประกาสค่าตัวแปรต่างๆ
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
from sklearn.cluster import KMeans

from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 1. Question or problem definition

1.ในปี 2019 ศิลปินคนไหนมีอันดับความนิยมสุงสุด 2.ในปี 2019 คนชอบฟังเพลงแนวไหนมากที่สุด 3.จากข้อมูลนี้สารมารถแบ่งแนวเพลงได้กี่กลุ่ม

# 2. Acquire training and testing data

In [ ]:
# import ข้อมูลเข้า notebook
filename='/kaggle/input/top-spotify-songs-from-20102019-by-year/top10s.csv'
df=pd.read_csv(filename,encoding='ISO-8859-1')
# แสดงข้อมูล 603 ข้อ จาก 0 - 602
df.head(603) 

# 3. Wrangle, prepare, cleanse the data

เช็คว่าข้อมูลสูญหายหรือไม่

In [ ]:
df = df.rename(columns = {'Unnamed: 0': 'id'})
df.head()

ดูผล ค่าที่ขึ้น false คือข้อมูลมีค่าครบ ไม่สูญหาย 

In [ ]:
#จากการทำการตรวจสอบถ้าข้อมูลขึ้น false จะแสดงได้ว่าข้อมูลถูกเคลียร์เรียบร้อยแล้วสามารถนำข้อมูลมาใช้ได้ต่อ
df.isnull().any()

In [ ]:
#จำนวนข้อมูลของ columns 
df = df.drop_duplicates()
df.shape

603 เป็น columns ของจำนวนเพลงติดอันดับตั้งแต่ 2010-2019
15 เป็น columns ของหมวดข้อมูลที่ใช้ในการเก็บ

In [ ]:
#การ drop ข้อมูลทิ้ง ใน columns id
df = df.drop(['id'], axis=1)
df.head()

In [ ]:
#ศิลปินที่ร้องเพลงที่คนนิยมในช่วง 10 ปี 
df['artist'].value_counts()

In [ ]:
#เพลงของศิลปิน Justin Bieber ที่ติดอันดับภายในปี 2010 - 2019 ติด top ในปี 2015-2016
df[df.title == 'Company']

In [ ]:
#แนวเพลงที่มีการเข้าฟังมากที่สุด  
df['top genre'].value_counts()

# 4. Analyze, identify patterns, and explore the data

In [ ]:
#จะได้ตารางที่เหลือแต่ข้อมูลที่เราจะนำไปเทรนโมเดล
yearless_df = df.drop(['year', 'title', 'top genre', 'artist'], axis=1)
yearless_df.drop_duplicates()

# 5. Model, predict and solve the problem

เป็นการเทรนตัว model

In [ ]:
#การทำ clustering ด้วย KMeans
#k = กลุ่มที่เราต้องการจะจับ
#sse = ระยะห่างของ k หรือความห่างของแต่ละกลุ่ม
from sklearn.cluster import KMeans

sse = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(yearless_df)
    yearless_df["clusters"] = kmeans.labels_
    #print(data["clusters"])
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()

Error Sum of Squares (SSE) คือ การหาผลรวมกำลังสองความคลาดเคลื่อน

In [ ]:
#การทำ kmeans โดยแบ่งข้อมูลเป็น 10 กลุ่ม ตั้งแต่ 0 - 9
kmeans = KMeans(n_clusters=10, max_iter=1000).fit(yearless_df)
df["clusters"] = kmeans.labels_
df.groupby(by=['clusters']).mean()

# 6.Visualize, report, and present the problem solving steps and final solution

In [ ]:
df.head(603)

การแสดงแบบ PairPlot Graph ทำให้การแสดงข้อมูลที่ออกมาอย่างหลากหลายและครบถ้วน โดยจะมีการแบ่งเพลงเป็นสีที่ต่างกัน เพื่อทำให้กราฟถูกแสดงอย่างครบถ้วน



In [ ]:
sns.pairplot(df, hue = 'clusters', diag_kind = 'kde',
             plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
             size = 4)
 


In [ ]:
sns.pairplot(df, vars=['bpm','nrgy','dnce','dB', 'live', 'dur', 'val','acous','spch','pop'],
            hue = 'clusters');